# 1. 패키지 설치

In [ ]:
%pip install langchain-openai python-dotenv

In [79]:
from dotenv import load_dotenv
load_dotenv()

True

In [80]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [81]:
llm.invoke("소득세법에 대해 알려줘")

AIMessage(content='소득세법은 국내에서 발생하는 소득에 대해 과세하는 법률입니다. 국가의 재정 수입원 중 하나인 소득세는 근로 소득, 사업 소득, 이자 소득, 배당소득 등 다양한 소득에 부과됩니다.\n\n소득세를 계산할 때는 해당 소득과 관련된 공제 항목을 고려하여 실제로 납부할 세액을 결정하게 됩니다. 또한 소득세율은 소득의 양에 따라 차등적으로 적용되며, 일정 기준 이하의 소득에는 비과세 지정이나 세액 감면 혜택을 받을 수 있는 경우도 있습니다.\n\n소득세법은 국가의 재정 운영을 위한 중요한 법률로, 소득 세액을 신고하고 납부하는 과정에서는 법률에 따라 세무 당국의 지침을 준수해야 합니다.세법에 따른 세무 행정이나 세무청의 세무조사에 부합하지 않을 경우 벌금이나 형사 처벌을 받을 수도 있으니 주의가 필요합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 374, 'prompt_tokens': 22, 'total_tokens': 396, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BdW3c7pzdFBqUmqrZgi0O0nFzavje', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7df6e8cf-3750-46bb-87da-bd5bf5a8b5fb-0', usage

In [82]:
%pip install --upgrade --quiet  docx2txt
%pip install --upgrade --quiet  docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install langchain-chroma

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [3.4 LangChain을 활용한 Vector Database 변경 (Chroma ➡️ Pinecone)](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.4%20LangChain%EC%9D%84%20%ED%99%9C%EC%9A%A9%ED%95%9C%20Vector%20Database%20%EB%B3%80%EA%B2%BD%20(Chroma%20%E2%9E%A1%EF%B8%8F%20Pinecone).ipynb)와 동일함

1. 문서의 내용을 읽는다.

2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- input 문서가 길면 답변 생성이 오래 걸림

In [83]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./docs/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [84]:
len(document_list)

915

### 3. 임베딩 -> 벡터데이터베이스에 저장

In [105]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(
    model='text-embedding-3-small',
    chunk_size=500
    )

## Chroma -> PineCone으로 벡터 데이터베이스 변경해보기
* LangChain 활용 시 벡터 데이터베이스 변경하는 작업이 엄청나게 간단해짐!

In [86]:
%pip install --upgrade --quiet langchain-pinecone langchain-openai langchain
%pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [106]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from tqdm import tqdm

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

index_name = pc.Index("tax-index-small")


# 벡터스토어 객체 준비
database = PineconeVectorStore(
    index=index_name,
    embedding=embedding,
    text_key="text"
)

# ✅ 안전한 소량 업로드
batch_size = 10  # 또는 5~20 사이에서 조정

for i in tqdm(range(0, len(document_list), batch_size)):
    batch = document_list[i:i+batch_size]
    texts = [doc.page_content for doc in batch]
    metadatas = [doc.metadata for doc in batch]
    try:
        database.add_texts(texts=texts, metadatas=metadatas)
    except Exception as e:
        print(f"❌ Batch {i}-{i+batch_size} 업로드 실패: {e}")

100%|██████████| 92/92 [01:48<00:00,  1.18s/it]


In [107]:
query = '연봉 5천만원인 거주자의 종합소득세는?'
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='fbeff489-2d1f-42f6-85e9-265fcf76bb1d', metadata={'source': './docs/tax_with_table.docx'}, page_content='제6관 종합소득공제 <개정 2009. 12. 31.>\n\n\n\n제50조(기본공제) ① 종합소득이 있는 거주자(자연인만 해당한다)에 대해서는 다음 각 호의 어느 하나에 해당하는 사람의 수에 1명당 연 150만원을 곱하여 계산한 금액을 그 거주자의 해당 과세기간의 종합소득금액에서 공제한다. <개정 2015. 12. 15.>\n\n1. 해당 거주자\n\n2. 거주자의 배우자로서 해당 과세기간의 소득금액이 없거나 해당 과세기간의 소득금액 합계액이 100만원 이하인 사람(총급여액 500만원 이하의 근로소득만 있는 배우자를 포함한다)\n\n3. 거주자(그 배우자를 포함한다. 이하 이 호에서 같다)와 생계를 같이 하는 다음 각 목의 어느 하나에 해당하는 부양가족(제51조제1항제2호의 장애인에 해당되는 경우에는 나이의 제한을 받지 아니한다)으로서 해당 과세기간의 소득금액 합계액이 100만원 이하인 사람(총급여액 500만원 이하의 근로소득만 있는 부양가족을 포함한다)'),
 Document(id='8638a6c2-7844-42e4-afc3-27611f539860', metadata={'source': './docs/tax_with_table.docx'}, page_content='제6관 종합소득공제 <개정 2009. 12. 31.>\n\n\n\n제50조(기본공제) ① 종합소득이 있는 거주자(자연인만 해당한다)에 대해서는 다음 각 호의 어느 하나에 해당하는 사람의 수에 1명당 연 150만원을 곱하여 계산한 금액을 그 거주자의 해당 과세기간의 종합소득금액에서 공제한다. <개정 2015. 12. 15.>\n\n1. 해당 거주자\n\n2. 거주자의 배우자로서 해당 과세기간의 소득금액이 없거나 해당 과세기간의 소득금액 합계액이 100만원 이하인 사람(총급여액 500만원 이하

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [108]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/woojung/.pyenv/versions/3.10.17/envs/inflearn-llm-application/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음
- 하단의 `dictionary_chain` 과 연계하여 사용

In [110]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [111]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [112]:
ai_message = qa_chain.invoke({"query": query})
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '죄송하지만, 연봉 5천만원인 거주자의 종합소득세를 계산하기 위한 충분한 세율과 공제 정보가 제공되지 않았습니다. 구체적인 소득세율과 공제 사항을 바탕으로 계산해야 하며, 이에 대한 정보는 주어진 문맥에 포함되어 있지 않습니다. 추가적인 세율표와 공제 기준을 확인해야 정확한 답변이 가능합니다.'}

langchain hub: 제공되는 prompt를 쓰기 용이함

In [94]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [95]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/woojung/.pyenv/versions/3.10.17/envs/inflearn-llm-application/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [96]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [97]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt": prompt}
)

In [98]:
ai_message = qa_chain({"query": query})

/var/folders/jf/33cgwln56pl8ksd6k07kdwsr0000gn/T/ipykernel_6393/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [99]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '죄송하지만 제시된 문맥에서는 연봉 5천만 원인 거주자의 종합소득세를 정확히 계산할 수 있는 정보가 없습니다. 종합소득세는 과세표준에 따른 세율 및 공제를 고려하여 구체적으로 계산되어야 합니다. 자세한 소득세 계산은 국세청 등의 공식 자료나 전문가의 상담을 통해 확인하는 것이 좋습니다.'}

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [113]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [115]:
new_question = dictionary_chain.invoke({"question": query})

In [117]:
ai_response = tax_chain.invoke({"question": query})

In [118]:
ai_response

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': "I don't know. The provided context does not include specific information regarding the calculation of comprehensive income tax for a resident with an annual income of 50 million won."}